<a href="https://colab.research.google.com/github/ehddnr301/dacon_cv2/blob/master/dacon_cv2_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Feb 19 08:39:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os

os.listdir('/content/drive/MyDrive/dacon_computer_vision/')

['data',
 'Untitled1.ipynb',
 'model.h5',
 'eff_model.pt',
 'eff_model1.pt',
 'dacon-vision2.ipynb',
 'Untitled0.ipynb',
 'checkpoint',
 'dacon_cv.zip',
 'eff_model_cv2.pt',
 'Untitled3.ipynb',
 'Untitled2.ipynb',
 'dacon_cv2.ipynb',
 'eff_model_0214.pt',
 'dacon_computervision2.ipynb',
 'resnet_model_0218.pt',
 'dacon_cv2_resnet.ipynb']

In [4]:
# colab 에서 자꾸 unzip한 이미지중에 일부가 없어져서 -n modifier를 주고 계속 실행하려함.
!unzip -n '/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist_2nd.zip' -d '/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist'

Archive:  /content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist_2nd.zip
error:  cannot create /content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist/00000.png
        Input/output error


In [5]:
!pip install torchinfo

In [6]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary

import albumentations


from torchvision import transforms


In [7]:
# random seed
random_seed = 777
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
EPOCHS=50

PATH_TRAIN_DATASET='/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist/'
PATH_TEST_DATASET='/content/drive/MyDrive/dacon_computer_vision/data/test_dirty_mnist/'
PATH_TRAIN_ANS_CSV='/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist_2nd_answer.csv'

In [8]:
df = pd.read_csv(PATH_TRAIN_ANS_CSV)
df.head()

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,2,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,3,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,4,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0


In [9]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.05, random_state=random_seed)

In [10]:
train.head()

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
31245,31245,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0
35091,35091,0,1,1,0,0,1,1,1,0,0,1,1,0,0,0,0,1,1,1,1,0,0,0,1,0,1
9730,9730,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0,1,1,1,1
28408,28408,0,1,1,0,0,0,1,1,0,1,1,1,0,1,1,0,0,1,0,1,1,0,0,0,0,1
43447,43447,0,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1


In [11]:
PATH_TRAIN_CSV='/content/drive/MyDrive/dacon_computer_vision/data/mnist_train.csv'
PATH_VALID_CSV='/content/drive/MyDrive/dacon_computer_vision/data/mnist_valid.csv'


In [12]:
train.to_csv(PATH_TRAIN_CSV, index=False)

In [13]:
valid.to_csv(PATH_VALID_CSV, index=False)

In [14]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = cv2.imread(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png'))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            augmented = self.transforms(image=image) 
            image = augmented['image']

        return image, target

In [15]:
from albumentations.pytorch import ToTensor

In [16]:
transforms_train = albumentations.Compose([
    albumentations.OneOf([
                          albumentations.HorizontalFlip(p=1),
                          albumentations.VerticalFlip(p=1),
                          albumentations.RandomRotate90(p=0.5)
    ], p=0.8),
    albumentations.OneOf([
                          albumentations.RandomBrightness(limit=0.1, p=1),
                          albumentations.RandomContrast(limit=0.1, p=1),
    ], p=0.7),
    albumentations.OneOf([
                          albumentations.Blur(p=1),
                          albumentations.OpticalDistortion(p=1),
                          albumentations.GaussNoise(p=1)                 
    ], p=0.3),
    ToTensor((
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )),
])

transforms_valid = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    ToTensor((
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )),
])

transforms_test = albumentations.Compose([
    ToTensor((
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )),
])

In [17]:
trainset = MnistDataset(PATH_TRAIN_DATASET, PATH_TRAIN_CSV, transforms_train)
valset = MnistDataset(PATH_TRAIN_DATASET, PATH_VALID_CSV, transforms_valid)
testset = MnistDataset(PATH_TEST_DATASET, '/content/drive/MyDrive/dacon_computer_vision/data/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=128)
val_loader = DataLoader(valset, batch_size=32)
test_loader = DataLoader(testset, batch_size=32)

In [18]:
# !pip install nfnets-pytorch

In [19]:
from torchvision.models import resnet50

# from nfnets import replace_conv

In [20]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel()
# replace_conv(model)

model = model.to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet: 1-1                            [1, 1000]                 --
|    └─Conv2d: 2-1                       [1, 64, 128, 128]         9,408
|    └─BatchNorm2d: 2-2                  [1, 64, 128, 128]         128
|    └─ReLU: 2-3                         [1, 64, 128, 128]         --
|    └─MaxPool2d: 2-4                    [1, 64, 64, 64]           --
|    └─Sequential: 2-5                   [1, 256, 64, 64]          --
|    |    └─Bottleneck: 3-1              [1, 256, 64, 64]          75,008
|    |    └─Bottleneck: 3-2              [1, 256, 64, 64]          70,400
|    |    └─Bottleneck: 3-3              [1, 256, 64, 64]          70,400
|    └─Sequential: 2-6                   [1, 512, 32, 32]          --
|    |    └─Bottleneck: 3-4              [1, 512, 32, 32]          379,392
|    |    └─Bottleneck: 3-5              [1, 512, 32, 32]          280,064
|    |    └─Bottleneck: 3-6              [1, 512, 32, 32] 

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min',
                                                  factor=0.5,
                                                  patience=2,)



for epoch in range(EPOCHS):
    

    for i, (images, targets) in enumerate(train_loader):

        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        model.train()

        with torch.set_grad_enabled(True):
            outputs = model(images)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

        if i % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch+1}: {loss.item():.5f}, {acc.item():.5f}')


    valid_acc_list = []
    with tqdm(val_loader,
            total=val_loader.__len__(),
            unit="batch") as valid_bar:
        for i, (images, targets) in enumerate(valid_bar):
            valid_bar.set_description(f"Valid Epoch {epoch + 1}")
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            # 모델의 dropoupt, batchnormalization를 eval모드로 설정
            # .forward()에서 중간 노드의 gradient를 계산

            model.eval()

            with torch.no_grad():
                # validation loss만을 계산
                probs  = model(images)
                valid_loss = criterion(probs, targets)


                # train accuracy 계산
                probs  = probs.cpu().detach().numpy()
                targets = targets.cpu().detach().numpy()
                preds = probs > 0.5
                batch_acc = (targets == preds).mean()
                valid_acc_list.append(batch_acc)

            valid_acc = np.mean(valid_acc_list)
            valid_bar.set_postfix(valid_loss = valid_loss.item(),
                                  valid_acc = valid_acc)

    scheduler.step(valid_loss)


    if (epoch + 1) % 5 == 0:
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f'/content/drive/MyDrive/dacon_computer_vision/checkpoint/model_resnet_0218_{epoch + 1}.pt')
    
    print('------------------')

1: 0.79138, 0.51713
1: 0.70026, 0.54657
1: 0.69251, 0.54026
1: 0.69292, 0.53335
1: 0.69730, 0.53966
1: 0.69568, 0.54357
1: 0.69692, 0.53035
1: 0.69315, 0.53726
1: 0.69182, 0.53726
1: 0.69051, 0.53906
1: 0.69290, 0.54026
1: 0.69402, 0.53876
1: 0.69156, 0.54177
1: 0.69313, 0.53666
1: 0.69286, 0.53005
1: 0.69230, 0.52945
1: 0.69281, 0.53636
1: 0.69138, 0.53516
1: 0.69079, 0.53966
1: 0.69104, 0.53756
1: 0.69037, 0.53996
1: 0.69135, 0.53786
1: 0.69038, 0.53936
1: 0.69078, 0.54627
1: 0.69088, 0.53546
1: 0.69225, 0.53666
1: 0.69000, 0.54117
1: 0.68891, 0.54688
1: 0.69094, 0.54087
1: 0.69111, 0.53666
1: 0.68994, 0.54177
1: 0.68793, 0.54928
1: 0.69009, 0.53726
1: 0.69124, 0.53245
1: 0.69101, 0.53606
1: 0.68916, 0.53966
1: 0.68798, 0.53846
1: 0.68847, 0.54026


Valid Epoch 1: 100%|██████████| 79/79 [09:25<00:00,  7.16s/batch, valid_acc=0.538, valid_loss=0.683]


------------------
2: 0.68926, 0.53966
2: 0.68829, 0.54868
2: 0.68776, 0.54026
2: 0.68862, 0.53275
2: 0.68631, 0.54207
2: 0.68779, 0.54417
2: 0.68913, 0.52855
2: 0.68763, 0.53906
2: 0.68826, 0.53786
2: 0.68808, 0.53936
2: 0.68740, 0.54026
2: 0.68686, 0.53996
2: 0.68693, 0.54117
2: 0.68734, 0.53726
2: 0.69034, 0.53125
2: 0.68842, 0.53005
2: 0.68725, 0.53816
2: 0.68842, 0.53576
2: 0.68601, 0.54026
2: 0.68728, 0.53726
2: 0.68660, 0.53996
2: 0.68671, 0.53846
2: 0.68578, 0.53966
2: 0.68619, 0.54627
2: 0.68792, 0.53425
2: 0.68781, 0.53666
2: 0.68553, 0.54477
2: 0.68534, 0.54688
2: 0.68851, 0.54117
2: 0.68782, 0.53666
2: 0.68544, 0.54477
2: 0.68274, 0.55349
2: 0.68522, 0.53996
2: 0.68672, 0.53546
2: 0.68864, 0.53726
2: 0.68140, 0.54597
2: 0.68395, 0.54447


  0%|          | 0/79 [00:00<?, ?batch/s]

2: 0.68586, 0.54297


...

------------------
11: 0.48755, 0.75661
11: 0.49911, 0.75180
11: 0.49562, 0.75090
11: 0.47678, 0.75811
11: 0.46350, 0.77043
11: 0.47598, 0.76352
11: 0.48265, 0.75451
11: 0.47136, 0.76442
11: 0.44741, 0.77554
11: 0.47479, 0.75901
11: 0.46590, 0.76803
11: 0.48673, 0.75631
11: 0.47482, 0.76322
11: 0.46161, 0.76893
11: 0.46357, 0.77224
11: 0.47445, 0.74910
11: 0.47322, 0.76112
11: 0.45719, 0.76412
11: 0.47517, 0.75841
11: 0.44558, 0.77825
11: 0.47577, 0.77284
11: 0.47704, 0.76172
11: 0.46199, 0.76863
11: 0.44983, 0.78245
11: 0.44872, 0.77764
11: 0.45215, 0.77855
11: 0.45016, 0.78365
11: 0.45784, 0.77494
11: 0.46519, 0.76683
11: 0.46348, 0.76773
11: 0.45618, 0.77614
11: 0.44578, 0.78606
11: 0.44637, 0.77734
11: 0.45065, 0.78155
11: 0.44647, 0.77794
11: 0.43930, 0.78606
11: 0.46321, 0.76713


  0%|          | 0/79 [00:00<?, ?batch/s]

11: 0.45041, 0.78395


Valid Epoch 11: 100%|██████████| 79/79 [00:16<00:00,  4.85batch/s, valid_acc=0.759, valid_loss=0.525]


------------------
12: 0.44965, 0.78756
12: 0.46574, 0.76743
12: 0.46085, 0.77254
12: 0.45911, 0.77103
12: 0.44939, 0.78516
12: 0.45842, 0.77524
12: 0.46128, 0.76683
12: 0.44676, 0.78185
12: 0.43446, 0.78726
12: 0.44427, 0.77584
12: 0.44096, 0.78816
12: 0.43830, 0.78456
12: 0.45142, 0.77885
12: 0.42606, 0.79988
12: 0.43575, 0.78305
12: 0.45721, 0.75931
12: 0.43260, 0.78996


In [ ]:
!nvidia-smi

In [ ]:
torch.save(model, '/content/drive/MyDrive/dacon_computer_vision/resnet_model_0218.pt')